<a href="https://colab.research.google.com/github/b202113003-tech/CodeReviewExercise/blob/main/ad_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 模型訓練

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

data_path = "train_Data.xlsx"
df = pd.read_excel(data_path)

# Select the relevant columns and then rename them
df = df[['content_cleaned', 'label']]
df.columns = ["text", "label"]

df['label'] = pd.to_numeric(df['label'], errors='coerce').fillna(-1).astype(int)
df = df[df['label'].isin([0,1])]

X = df["text"].astype(str).values
y = df["label"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline = Pipeline([
    ("hash", HashingVectorizer(analyzer="char", ngram_range=(2,4), n_features=2**18, alternate_sign=False)),
    ("clf", SGDClassifier(loss="log_loss", max_iter=5, class_weight="balanced"))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

joblib.dump(pipeline, "ad_legal_model.joblib")
print("模型已儲存為 ad_legal_model.joblib")

Accuracy: 0.9611650485436893
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       155
           1       0.95      0.97      0.96       154

    accuracy                           0.96       309
   macro avg       0.96      0.96      0.96       309
weighted avg       0.96      0.96      0.96       309

模型已儲存為 ad_legal_model.joblib


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_stochastic_gradient.py:738: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
